In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time # to be used to delay
import re
import glob
import requests
import json

In [2]:
# # Set some default behaviors for our browser
# options = webdriver.ChromeOptions()

# # Make sure notifications are off
# options.add_argument("--disable-notifications")

# # executable_path = {'executable_path':'resources/chromedriver'}
# executable_path = {'executable_path':ChromeDriverManager().install()}

# # create a new browser object by default it is firefox the default one
# browser = Browser('chrome', **executable_path, headless = False, options = options) #headless true will not open browser

In [3]:
msa_df = pd.read_csv("data/final_msa_formatted_ordered.csv")
msa_df.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,1985432,"April 24, 2021",Louisiana,New Orleans,100 block of Bourbon St,0,5,View Incident View Source
1,1984807,"April 22, 2021",California,San Diego,500 block of J St,1,4,View Incident View Source
2,1981805,"April 19, 2021",Texas,Houston,700 E Whitney St,2,4,View Incident View Source
3,1982260,"April 19, 2021",Georgia,Atlanta,1191 Oakland Ln SW,0,5,View Incident View Source
4,1982020,"April 18, 2021",Alabama,Montgomery,2300 block of Day St,1,4,View Incident View Source


In [ ]:
# for index, row in msa_df.iterrows():
#     street = row['Address']
#     city = row['City Or County']
#     state = row['State']
    
#     address = f"{street}, {city}, {state}"
#     print(address)
    

In [ ]:
# coord_url = 'https://www.latlong.net/convert-address-to-lat-long.html'
# browser.visit(coord_url)
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')
# input_id = soup.find("input")
# current_id = input_id["id"] #This value changes everytime page is loaded but not when a button is clicked
# address_box = browser.find_by_id(current_id)
# address_box.fill("500 block of J St, San Diego, California")
# browser.click_link_by_id("btnfind")
# time.sleep(5)
# latlng = browser.find_by_id("latlngspan")


In [ ]:
# # Did not use due to website limiting 10 results per day. It blocked my access once they are reached
# coord_url = 'https://www.latlong.net/convert-address-to-lat-long.html'
# browser.visit(coord_url)
# msa_df['coordinates'] = ""
# for index, row in msa_df.iterrows():
#     street = row['Address']
#     city = row['City Or County']
#     state = row['State']
#     address = f"{street}, {city}, {state}"
    
#     html = browser.html
#     soup = BeautifulSoup(html, 'html.parser')
#     try:
#         browser.click_link_by_partial_text("CLOSE [X]")
#     except:
#         pass
#     input_id = soup.find("input")
#     current_id = input_id["id"] #This value changes everytime page is loaded but not when a button is clicked
#     address_box = browser.find_by_id(current_id)
#     try:
#         browser.click_link_by_partial_text("CLOSE [X]")
#     except:
#         pass
#     address_box.fill(address)
#     try:
#         browser.click_link_by_partial_text("CLOSE [X]")
#     except:
#         pass
#     browser.click_link_by_id("btnfind")
#     time.sleep(5)
#     latlng = browser.find_by_id("latlngspan")
#     coord = latlng.text
    
#     msa_df.loc[index, 'coordinates'] = coord

In [ ]:
# msa_df.head(15)

# Google API

In [ ]:
# #--------------------------------------------------#
# #                Google Geocode API                #
# #--------------------------------------------------#

# target_city = "Seattle, Washington"
# # 500 block of J St, San Diego, California
# gkey = ""Enter Google API Key""

# params = {
#         "address": target_city, 
#         "key": gkey
# }

# # Build URL using the Google Maps API
# base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# # Run request
# response = requests.get(base_url, params=params)
# response_json = response.json()

# lat = response_json["results"][0]["geometry"]["location"]["lat"]
# lng = response_json["results"][0]["geometry"]["location"]["lng"]

# print(response.url)

# # Print the json (pretty printed)
# print(json.dumps(response_json, indent=4, sort_keys=True))

In [7]:
def get_lanlng(df):
    df['coordinates'] = ""
    df['latitude'] = ""
    df['longitude'] = ""
    gkey = "Enter Google API Key"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    for index, row in df.iterrows():
        street = row['Address']
        city = row['City Or County']
        state = row['State']
        address = f"{street}, {city}, {state}"
        params = {
            "address": address, 
            "key": gkey
    }
    #     target_address = "Seattle, Washington"
        response = requests.get(base_url, params=params)
        response_json = response.json()
        try:
            lat = response_json["results"][0]["geometry"]["location"]["lat"]
            lng = response_json["results"][0]["geometry"]["location"]["lng"]
            coord = f"{lat}, {lng}"

            df.loc[index, 'coordinates'] = coord
            df.loc[index, 'latitude'] = lat
            df.loc[index, 'longitude'] = lng
        except:
            pass
    return df


In [8]:
msa_df_updated = get_lanlng(msa_df)
msa_df_updated.head(60)

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,coordinates,latitude,longitude
0,1985432,"April 24, 2021",Louisiana,New Orleans,100 block of Bourbon St,0,5,View Incident View Source,"29.9542063, -90.0691878",29.9542,-90.0692
1,1984807,"April 22, 2021",California,San Diego,500 block of J St,1,4,View Incident View Source,"32.7095105, -117.1599882",32.7095,-117.16
2,1981805,"April 19, 2021",Texas,Houston,700 E Whitney St,2,4,View Incident View Source,"29.8304693, -95.3892745",29.8305,-95.3893
3,1982260,"April 19, 2021",Georgia,Atlanta,1191 Oakland Ln SW,0,5,View Incident View Source,"33.7197359, -84.42625249999999",33.7197,-84.4263
4,1982020,"April 18, 2021",Alabama,Montgomery,2300 block of Day St,1,4,View Incident View Source,"32.3698089, -86.3293563",32.3698,-86.3294
5,1981521,"April 18, 2021",Michigan,Detroit,Ward Ave and W McNichols Rd,1,5,View Incident View Source,"42.4166243, -83.1742467",42.4166,-83.1742
6,1981353,"April 18, 2021",Louisiana,Shreveport,3701 Hearne Ave,0,6,View Incident View Source,"32.4777776, -93.7795905",32.4778,-93.7796
7,1980689,"April 18, 2021",Wisconsin,Kenosha,1548 Sheridan Rd,3,3,View Incident View Source,"42.6303111, -87.82177659999999",42.6303,-87.8218
8,1980492,"April 17, 2021",Ohio,Columbus,5001 Chatterton Rd,1,5,View Incident View Source,"39.914772, -82.86195909999999",39.9148,-82.862
9,1980971,"April 17, 2021",Louisiana,La Place (Laplace),624 Golfview Dr,0,9,View Incident View Source,"30.0843142, -90.4843125",30.0843,-90.4843


In [9]:
# msa_df_updated.head(60)

In [12]:
len(msa_df_updated)

5925

In [13]:
cleaned_msa_df = msa_df_updated.dropna(how='any')

In [14]:
len(cleaned_msa_df)

5916

In [15]:
cleaned_msa_df.to_csv("data/final_msa_with_coordinates.csv")